In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
  return false;
}

<IPython.core.display.Javascript object>

In [2]:
# Modules
import scipy as sp
from scipy import sparse, signal, stats
from astropy import convolution as conv
import numpy as np
from numpy.fft import fft, ifft
import matplotlib.pyplot as plt
from matplotlib import colors as clrs
import pandas as pd 
import time
import datetime
import calendar
import sqlite3
from tqdm.autonotebook import tqdm

%matplotlib inline 



# Color definitions
ClrS = (0.74, 0.00, 0.00)
ClrN = (0.20, 0.56, 1.00)

Clr = [(0.00, 0.00, 0.00),
      (0.31, 0.24, 0.00),
      (0.43, 0.16, 0.49),
      (0.32, 0.70, 0.30),
      (0.45, 0.70, 0.90),
      (1.00, 0.82, 0.67)]

# Font Size

font = {'family': 'sans-serif',
        'weight': 'normal',
        'size'   : 21}

plt.rc('font', **font)

C:\Users\Andres\AppData\Local\Temp\ipykernel_489364\1102906597.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


### Read files

In [3]:
CDELT1 = 0.504298
Rs = 953.517029
Rsun = 696000000
DOBS = 150559255066.71

life_cut = 10

id_offset = 0

features = {}
for i in range(10):
    time = 45*2**i

    Feature = pd.read_pickle(f'data/SWAMIS/100_frames/swamis_NoAssociate_{str(time).zfill(5)}s.pnds')

    Feature['id'] = Feature['id'] + id_offset
    id_offset = np.nanmax(Feature['id'])+1

    # Converting flux to maxwells
    Feature['Flux'] = Feature['flux']*(CDELT1*Feature['scale'])**2/(np.pi*Rs**2)*2*np.pi*(Rsun*100)**2*Feature['scale']
    # Feature['Flux'] = Feature['Flux']*(CDELT1)**2/(np.pi*Rs**2)*2*np.pi*(Rsun*100)**2

    # Add distance from disk center in arcseconds
    Feature['Dis'] = np.sqrt(Feature['xs']**2+Feature['ys']**2)
    Feature['Ang'] = -(Feature['Dis']/60/60*np.pi/180- np.arcsin(np.sin(Feature['Dis']/60/60*np.pi/180)*DOBS/Rsun))
    Feature['Area'] = 2*np.pi*(Rsun/1000000)**2*Feature['size_s']/(np.pi*Rs**2)*np.divide(1,np.cos(Feature['Ang']))
    Feature['meanB'] = np.divide(np.abs(Feature['Flux']),Feature['Area']*100000000**2)

    Feature = Feature.loc[Feature['Life']<np.nanmax(Feature['Life']), :]

    # print(np.nanmin(Feature['Life']))

    if i>0:
        Feature = Feature.loc[Feature['Life']>life_cut, :]

    # print(np.nanmin(Feature['Life']))

    Feature['Life'] = Feature['Life']*time/60/60
    Feature['Timeline'] = (Feature['frame'] - 1)*time/60/60

    # print(np.nanmin(Feature['Timeline']), np.nanmax(Feature['Timeline']))

    features[time] = {}

    features[time]['features'] = Feature

    

In [13]:
total_files = 0
for i in range(10):
    time = 45*2**i
    total_files = total_files + features[time]['features'].shape[0]
    print(total_files, features[time]['features'].shape[0])


1481527 1481527
2103355 621828
2671089 567734
3161157 490068
3513590 352433
3747372 233782
3918179 170807
4030860 112681
4095902 65042
4129822 33920


In [4]:
Feature

,id,frame,scale,assoc_larger,flux,maxB,str_pix,size,x,y,...,lon,lat,size_s,Life,Flux,Dis,Ang,Area,meanB,Timeline
1890,6979674,1,1,NaN,-24778.196256,-476.013032,41,81,3161.742436,2884.034399,...,-42.879541,-29.519664,20.599634,76.8,-67148412202375094272.0,705.652342,0.829729,32.516095,206.508227,0.0
2571,6980438,1,1,NaN,-64521.365495,-905.031472,88,245,2878.417008,2599.141375,...,-28.253694,-20.907779,62.307536,121.6,-174851599422940479488.0,506.118756,0.557082,78.221480,223.533995,0.0
4118,6982174,1,1,NaN,-4491.352396,-393.693652,5,26,2924.535307,1481.825638,...,-28.606158,13.366641,6.612228,70.4,-12171474426118823936.0,529.756074,0.586495,8.459682,143.876258,0.0
4513,6982646,1,1,NaN,-42867.267494,-663.961651,50,215,1390.327709,1273.980122,...,21.348733,19.768605,54.678042,89.6,-116169430492867444736.0,509.507008,0.561264,68.823218,168.793953,0.0
5732,6983922,1,1,NaN,42052.598714,596.568724,60,191,1818.681150,1221.395660,...,7.207341,21.243039,48.574446,83.2,113961694526007050240.0,431.495215,0.467510,57.982537,196.544857,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1257461,8196426,95,8,NaN,1119.109145,130.740571,5,8,462.418774,201.612515,...,64.392151,-13.978702,130.533637,70.4,1552774766773474951168.0,859.030760,1.117693,317.745935,488.684385,601.6
1257462,8196431,95,8,NaN,1146.731922,205.126907,3,16,157.040746,209.133992,...,-25.696877,-12.924634,261.067274,70.4,1591101636933764775936.0,443.373464,0.481472,313.874741,506.922485,601.6
1257463,8196495,95,8,NaN,286.700949,63.871381,0,6,195.768495,318.511323,...,-15.422196,13.816503,97.900228,70.4,397800341143339401216.0,354.554412,0.379271,112.302665,354.22164,601.6
1257464,8196513,95,8,NaN,571.332337,116.992035,2,8,224.341656,353.118555,...,-8.544484,22.738559,130.533637,70.4,792729145437188653056.0,416.218975,0.449699,154.451663,513.253874,601.6


### Test for cleaning common elements

In [5]:
combined_features = features[23040]['features'].copy()
time2clean = 11520
features2clean = features[time2clean]['features'].copy()

unique_id = np.unique(combined_features['id'])

i = 0
snglFtr = combined_features[combined_features['id'] == unique_id[i]].copy().reset_index(drop=True)

matching_mask = np.logical_and(features2clean['Timeline']>=np.min(snglFtr['Timeline']), features2clean['Timeline']<=np.max(snglFtr['Timeline']))
matching_frames = features2clean.loc[matching_mask].copy().reset_index(drop=True)
matching_frames['detector dis'] = np.nan
matching_frames['detector range'] = np.nan
matching_frames['size dif'] = np.nan
matching_frames['match id'] = np.nan



In [6]:
snglFtr = snglFtr.set_index(['Timeline'])
snglFtr

,id,frame,scale,assoc_larger,flux,maxB,str_pix,size,x,y,...,ys,lon,lat,size_s,Life,Flux,Dis,Ang,Area,meanB
Timeline,,,,,,,,,,,,,,,,,,,,,
0.0,6979674,1,1,NaN,-24778.196256,-476.013032,41,81,3161.742436,2884.034399,...,-420.918343,-42.879541,-29.519664,20.599634,76.8,-67148412202375094272.0,705.652342,0.829729,32.516095,206.508227
6.4,6979674,2,1,NaN,-21657.839110,-441.101639,32,93,3088.488903,2880.463629,...,-419.150578,-39.524035,-29.542817,23.651901,76.8,-58692307259963932672.0,675.314259,0.783723,35.583305,164.943384
12.8,6979674,3,1,NaN,-21503.517769,-464.494413,36,86,3007.824422,2874.437088,...,-416.111538,-35.940914,-29.475357,21.871997,76.8,-58274099536664870912.0,641.964374,0.735498,31.431901,185.39795
19.2,6979674,4,1,NaN,-46370.676230,-626.656178,72,172,2924.749539,2876.592652,...,-417.268524,-32.522146,-29.686946,43.742607,76.8,-125663597522595266560.0,611.444537,0.693166,60.595693,207.380413
25.6,6979674,5,1,NaN,-89181.361925,-841.988938,119,303,2846.509560,2873.748677,...,-415.892548,-29.356869,-29.702461,77.057891,76.8,-241679692480811859968.0,582.173115,0.653938,103.455963,233.606344
32.0,6979674,6,1,NaN,-235405.240334,-898.675127,375,654,2744.143501,2878.591595,...,-418.345558,-25.417696,-29.963618,166.326271,76.8,-637943454375065681920.0,549.126639,0.611039,216.391796,294.809446
38.4,6979674,7,1,NaN,-218934.753789,-963.849546,311,718,2659.075304,2876.673048,...,-417.450040,-22.163924,-29.973075,182.605021,76.8,-593308767964094005248.0,521.596555,0.576278,232.061392,255.668883
44.8,6979674,8,1,NaN,-43119.043399,-921.887344,57,158,2553.346021,2884.739416,...,-421.566228,-18.314628,-30.336781,40.182162,76.8,-116851738118289227776.0,495.055782,0.543504,50.026573,233.579336
51.2,6979674,9,1,NaN,-18787.830383,-368.519251,22,102,2462.821821,2877.862814,...,-418.178230,-14.987652,-30.156551,25.940383,76.8,-50914641483295137792.0,469.717835,0.512818,31.722512,160.500032


In [7]:
timestamps = snglFtr.index.get_level_values(0).values

matching_frames = matching_frames.set_index(['Timeline']).loc[(timestamps, slice(None))]
matching_frames

,id,frame,scale,assoc_larger,flux,maxB,str_pix,size,x,y,...,Life,Flux,Dis,Ang,Area,meanB,detector dis,detector range,size dif,match id
Timeline,,,,,,,,,,,,,,,,,,,,,
0.0,5821335,1,1,NaN,-371.372528,-76.479749,0,3,821.049868,2979.352351,...,38.4,-1006412062948050816.0,773.483737,0.942436,1.383069,72.766581,NaN,NaN,NaN,NaN
0.0,5822375,1,1,NaN,-27177.767172,-454.437930,43,57,2269.945787,3776.972907,...,35.2,-73651200999130308608.0,879.617466,1.170239,39.614362,185.920451,NaN,NaN,NaN,NaN
0.0,5822399,1,1,NaN,-3492.455066,-215.239622,5,14,2066.713946,3743.607000,...,35.2,-9464482804927834112.0,855.201342,1.108545,8.507346,111.250707,NaN,NaN,NaN,NaN
0.0,5822499,1,1,NaN,-30334.818085,-540.076182,46,88,1838.412619,3584.669951,...,35.2,-82206745310681923584.0,781.559976,0.957030,41.406041,198.538048,NaN,NaN,NaN,NaN
0.0,5822505,1,1,NaN,-4820.998589,-269.792121,8,19,2005.542618,3569.957955,...,41.6,-13064808961906161664.0,767.714915,0.932192,8.638099,151.246336,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70.4,6105544,23,8,NaN,382.360447,89.738278,1,6,205.667654,335.346339,...,144.0,530528819562415390720.0,382.582712,0.411009,113.795325,466.213195,NaN,NaN,NaN,NaN
70.4,6105546,23,8,NaN,2989.688276,193.861055,14,24,417.992821,345.254759,...,41.6,4148221409762310881280.0,746.530533,0.895733,667.693635,621.276165,NaN,NaN,NaN,NaN
70.4,6105554,23,8,NaN,94.017924,35.556245,0,2,109.000000,359.588767,...,35.2,130450778591374786560.0,729.348856,0.867358,53.757558,242.665003,NaN,NaN,NaN,NaN


In [8]:
for time in timestamps:
    xref = snglFtr.loc[(time,'xs')]
    yref = snglFtr.loc[(time,'ys')]
    rref = np.sqrt(snglFtr.loc[(time,'size_s')]/np.pi)

    matching_frames.loc[(time, 'detector dis')] = np.sqrt( (xref - matching_frames.loc[(time, 'xs')])**2 + (yref - matching_frames.loc[(time, 'ys')])**2 )
    matching_frames.loc[(time, 'detector range')] = rref
    matching_frames.loc[(time, 'size dif')] = 1-matching_frames.loc[(time, 'size_s')]/snglFtr.loc[(time,'size_s')]
    matching_frames.loc[(time, 'match id')] = snglFtr.loc[(time,'id')]


matching_frames

,id,frame,scale,assoc_larger,flux,maxB,str_pix,size,x,y,...,Life,Flux,Dis,Ang,Area,meanB,detector dis,detector range,size dif,match id
Timeline,,,,,,,,,,,,,,,,,,,,,
0.0,5821335,1,1,NaN,-371.372528,-76.479749,0,3,821.049868,2979.352351,...,38.4,-1006412062948050816.0,773.483737,0.942436,1.383069,72.766581,1181.384904,2.560677,0.962963,6979674.0
0.0,5822375,1,1,NaN,-27177.767172,-454.437930,43,57,2269.945787,3776.972907,...,35.2,-73651200999130308608.0,879.617466,1.170239,39.614362,185.920451,636.423365,2.560677,0.296296,6979674.0
0.0,5822399,1,1,NaN,-3492.455066,-215.239622,5,14,2066.713946,3743.607000,...,35.2,-9464482804927834112.0,855.201342,1.108545,8.507346,111.250707,702.035064,2.560677,0.827160,6979674.0
0.0,5822499,1,1,NaN,-30334.818085,-540.076182,46,88,1838.412619,3584.669951,...,35.2,-82206745310681923584.0,781.559976,0.957030,41.406041,198.538048,755.116497,2.560677,-0.086420,6979674.0
0.0,5822505,1,1,NaN,-4820.998589,-269.792121,8,19,2005.542618,3569.957955,...,41.6,-13064808961906161664.0,767.714915,0.932192,8.638099,151.246336,677.955308,2.560677,0.765432,6979674.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70.4,6105544,23,8,NaN,382.360447,89.738278,1,6,205.667654,335.346339,...,144.0,530528819562415390720.0,382.582712,0.411009,113.795325,466.213195,754.371445,2.110055,-5.998924,6979674.0
70.4,6105546,23,8,NaN,2989.688276,193.861055,14,24,417.992821,345.254759,...,41.6,4148221409762310881280.0,746.530533,0.895733,667.693635,621.276165,1064.944233,2.110055,-26.995697,6979674.0
70.4,6105554,23,8,NaN,94.017924,35.556245,0,2,109.000000,359.588767,...,35.2,130450778591374786560.0,729.348856,0.867358,53.757558,242.665003,990.912789,2.110055,-1.332975,6979674.0


In [9]:
matching_frames.loc[matching_frames['detector dis']<=matching_frames['detector range'],['xs','ys', 'scale', 'size', 'size_s']]

,xs,ys,scale,size,size_s
Timeline,,,,,
19.2,-445.132764,-419.421724,8,34,554.752832
32.0,-356.650940,-419.318401,8,28,456.860930
38.4,-313.376482,-417.622013,8,29,473.183021


In [10]:
snglFtr.loc[matching_frames.loc[matching_frames['detector dis']<=matching_frames['detector range'],:].index.get_level_values(0).values,['xs','ys', 'scale', 'size', 'size_s']]

,xs,ys,scale,size,size_s
Timeline,,,,,
19.2,-446.935566,-417.268524,1,172,43.742607
32.0,-355.706422,-418.345558,1,654,166.326271
38.4,-312.727406,-417.450040,1,718,182.605021


### Single file all features match

In [11]:
combined_features = features[23040]['features'].copy()
combined_features['origin'] = 23040

for i in tqdm(np.arange(8,-1,-1), dynamic_ncols=True, leave=True):
    time2clean = 45*2**i

    features2clean = features[time2clean]['features'].copy()
    features2clean['origin'] = time2clean

    all_match_candidates = None
    for unique_id in tqdm(np.unique(combined_features['id']), dynamic_ncols=True, leave=False):

        snglFtr = combined_features[combined_features['id'] == unique_id].copy().reset_index(drop=True)
        matching_mask = np.logical_and(features2clean['Timeline']>=np.min(snglFtr['Timeline']), features2clean['Timeline']<=np.max(snglFtr['Timeline']))
        matching_frames = features2clean.loc[matching_mask].copy().reset_index(drop=True)
        matching_frames['detector dis'] = np.nan
        matching_frames['detector range'] = np.nan
        matching_frames['size dif'] = np.nan
        matching_frames['match id'] = np.nan

        snglFtr = snglFtr.set_index(['Timeline'])
        timestamps = snglFtr.index.get_level_values(0).values
        timestamps = timestamps[timestamps<=np.max(matching_frames['Timeline'])]

        if timestamps.shape[0]>0:

            matching_frames = matching_frames.set_index(['Timeline']).loc[(timestamps, slice(None))]

            for time in timestamps:
                xref = snglFtr.loc[(time,'xs')]
                yref = snglFtr.loc[(time,'ys')]
                rref = np.sqrt(snglFtr.loc[(time,'size_s')]/np.pi)

                matching_frames.loc[(time, 'detector dis')] = np.sqrt( (xref - matching_frames.loc[(time, 'xs')])**2 + (yref - matching_frames.loc[(time, 'ys')])**2 )
                matching_frames.loc[(time, 'detector range')] = rref
                matching_frames.loc[(time, 'size dif')] = 1-matching_frames.loc[(time, 'size_s')]/snglFtr.loc[(time,'size_s')]
                matching_frames.loc[(time, 'match id')] = snglFtr.loc[(time,'id')]

            matching_mask = np.logical_and(matching_frames['detector dis']<=matching_frames['detector range'], np.abs(matching_frames['size dif'])<0.1)
            matching_frames = matching_frames.loc[matching_mask,:]

            if matching_frames.shape[0] > 0:
                if all_match_candidates is None:
                    all_match_candidates = matching_frames.copy()
                else:
                    all_match_candidates = pd.concat([all_match_candidates, matching_frames])

    indices2keep = np.setdiff1d(np.unique(features2clean['id']), np.unique(all_match_candidates['id']))
    features2keep = features2clean.set_index(['id']).loc[indices2keep, :].reset_index()

    combined_features = pd.concat([combined_features, features2keep]).sort_values(by=['Timeline']).reset_index(drop=True)

combined_features


  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/2086 [00:00<?, ?it/s]

  0%|          | 0/4477 [00:00<?, ?it/s]

  0%|          | 0/8513 [00:00<?, ?it/s]

  0%|          | 0/14073 [00:00<?, ?it/s]

...

  0%|          | 0/31585 [00:00<?, ?it/s]

  0%|          | 0/43727 [00:00<?, ?it/s]

  0%|          | 0/54191 [00:00<?, ?it/s]

  0%|          | 0/64094 [00:00<?, ?it/s]

,id,frame,scale,assoc_larger,flux,maxB,str_pix,size,x,y,...,lat,size_s,Life,Flux,Dis,Ang,Area,meanB,Timeline,origin
0,6979674,1,1,NaN,-24778.196256,-476.013032,41,81,3161.742436,2884.034399,...,-29.519664,20.599634,76.8000,-67148412202375094272.0,705.652342,0.829729,32.516095,206.508227,0.0,23040
1,4721,1,1,NaN,-1308.093986,-127.863934,1,6,3775.358275,1865.976990,...,3.770827,1.525899,0.0625,-3544908324376496128.0,880.097300,1.171543,4.182811,84.749421,0.0,45
2,4722,1,1,NaN,-5231.658119,-186.154446,9,21,353.026852,1866.404560,...,3.369492,5.340646,0.0375,-14177688006889373696.0,855.493124,1.109236,12.778744,110.947426,0.0,45
3,4726,1,1,NaN,-434.703320,-105.936842,0,5,2165.155590,1864.619452,...,0.960283,1.271582,0.0875,-1178037232347185664.0,112.052477,0.117244,1.364357,86.343796,0.0,45
4,4727,1,1,NaN,-430.486634,-121.253274,1,2,355.633634,1863.633634,...,3.441960,0.508633,0.0125,-1166610096101888256.0,854.337035,1.106504,1.210379,96.383851,0.0,45
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1905909,8196426,95,8,NaN,1119.109145,130.740571,5,8,462.418774,201.612515,...,-13.978702,130.533637,70.4000,1552774766773474951168.0,859.030760,1.117693,317.745935,488.684385,601.6,23040
1905910,8169018,95,8,NaN,-113.067123,-69.317365,0,2,290.253396,385.000000,...,31.625481,32.633409,83.2000,-156881727451008958464.0,541.338312,0.601120,42.165450,372.062261,601.6,23040
1905911,8155398,95,8,NaN,-1239.342069,-250.855769,4,12,393.448545,159.807899,...,-25.311679,195.800456,89.6000,-1719599111682337800192.0,673.873389,0.781592,293.949976,584.997194,601.6,23040
1905912,8182843,95,8,NaN,-2668.203877,-324.237786,9,24,395.530688,302.353119,...,10.080614,391.600911,76.8000,-3702158696371389988864.0,592.408739,0.667515,531.331036,696.770646,601.6,23040


In [12]:
combined_features.to_pickle('data\SWAMIS\\100_frames\swamis_all_combined_45-23040.pnds')

In [16]:
# Procesing objects

# Assembling pandas dataframe with columns
Feature = pd.DataFrame({'id':np.unique(combined_features['id'])
             })

for i in tqdm(np.arange(0,Feature.shape[0])):
# for i in np.arange(0,5):
    snglFtr = combined_features[combined_features['id'] == Feature['id'][i]].copy().reset_index(drop=True)
    Feature.loc[i,'Life'] = np.max(snglFtr['Timeline'])-np.min(snglFtr['Timeline'])
    
    maxSzin = (snglFtr['size_s'].values==np.max(snglFtr['size_s'].values)).nonzero()[0][0]
    
    Feature.loc[i,'scale'] = snglFtr.loc[maxSzin, 'scale']
    Feature.loc[i,'Flux'] = snglFtr.loc[maxSzin, 'flux']
    Feature.loc[i,'Lat'] = snglFtr.loc[maxSzin, 'lat']
    Feature.loc[i,'Lon'] = snglFtr.loc[maxSzin, 'lon']
    Feature.loc[i,'x'] = snglFtr.loc[maxSzin, 'x']
    Feature.loc[i,'xerr'] = snglFtr.loc[maxSzin, 'xerr']
    Feature.loc[i,'y'] = snglFtr.loc[maxSzin, 'y']
    Feature.loc[i,'yerr'] = snglFtr.loc[maxSzin, 'yerr']
    Feature.loc[i,'MaxSize'] = snglFtr.loc[maxSzin, 'size']
    Feature.loc[i,'xs'] = snglFtr.loc[maxSzin, 'xs']
    Feature.loc[i,'ys'] = snglFtr.loc[maxSzin, 'ys']
    Feature.loc[i,'size_s'] = snglFtr.loc[maxSzin, 'size_s']
    Feature.loc[i,'maxB'] = snglFtr.loc[maxSzin, 'maxB']
    Feature.loc[i,'Timeline'] = snglFtr.loc[maxSzin, 'Timeline']
    Feature.loc[i,'Origin'] = snglFtr.loc[maxSzin, 'origin']

    

Feature

  0%|          | 0/483832 [00:00<?, ?it/s]

,id,Life,scale,Flux,Lat,Lon,x,xerr,y,yerr,MaxSize,xs,ys,size_s,maxB,Timeline,Origin
0,1,0.075,1.0,-1756.405210,-73.460572,-14.117696,2168.980583,0.287084,3815.946495,0.308943,7.0,-66.316219,-891.486901,1.780222,-114.530262,0.05,45.0
1,3,0.000,1.0,-401.259592,-73.077809,-0.515724,2042.486745,0.499824,3810.000000,0.000086,2.0,-2.502384,-888.572155,0.508633,-74.707427,0.00,45.0
2,4,0.000,1.0,-744.679039,-72.768028,21.967109,1827.657345,0.335591,3809.082584,0.570318,3.0,105.836138,-888.242242,0.762949,-95.060790,0.00,45.0
3,5,0.000,1.0,-412.025671,-72.902785,16.705742,1877.470658,0.499138,3809.470658,0.499138,2.0,80.715163,-888.407168,0.508633,-77.163422,0.00,45.0
4,6,0.000,1.0,-935.759582,-72.790183,16.083357,1882.287636,0.261344,3807.993468,0.392628,4.0,78.286886,-887.659248,1.017266,-95.803365,0.00,45.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
483827,8196426,64.000,8.0,1240.328246,-14.212224,60.654953,455.364297,0.198822,201.135379,0.364870,11.0,802.715213,-218.675513,179.483995,83.694510,595.20,23040.0
483828,8196431,64.000,8.0,1417.217418,-12.101807,-44.927955,94.047507,0.578695,211.514801,0.252520,17.0,-656.746771,-176.761835,277.383439,126.062100,569.60,23040.0
483829,8196495,64.000,8.0,707.559467,13.522672,-33.939155,128.762686,0.193841,316.821372,0.376565,9.0,-516.518617,248.612492,146.850056,126.119683,569.60,23040.0
483830,8196513,64.000,8.0,1436.398568,22.650935,-36.866520,126.207875,0.329989,352.162515,0.272863,19.0,-526.810704,391.360386,310.017387,136.423972,556.80,23040.0


In [17]:
Feature.to_pickle('data\SWAMIS\\100_frames\swamis_all_combined_45-23040_SF.pnds')